In [1]:
!git clone https://github.com/bpuvaca/irony-detection-tar2024.git

Cloning into 'irony-detection-tar2024'...
remote: Enumerating objects: 841, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 841 (delta 48), reused 66 (delta 26), pack-reused 748 (from 1)
Receiving objects: 100% (841/841), 7.84 MiB | 9.79 MiB/s, done.
Resolving deltas: 100% (524/524), done.


In [5]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 9.5 MB/s eta 0:00:00


In [7]:

%cd irony-detection-tar2024/nemojte/
!python3 transformer_main.py --ds irony --model roberta



[Errno 2] No such file or directory: 'irony-detection-tar2024/nemojte/'
/content/irony-detection-tar2024/nemojte
Training roberta on irony
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Parsed dataset type train with 2148 tweets, 810 1s and 1338 0s
Parsed dataset type valid with 460 tweets, 173 